# Calculate Partial Derivatives of Phi to O2,T,S at O2=O2_bar, T=T_bar, S=S_bar

In [1]:
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']

### the partial derivatives

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_dPhidO2_dPhidT_dPhidS_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)

## dT, dS, dO2, dAOU

In [4]:
filename = f'{path}/DPLE-FOSI_hindcast_dO2_dT_dS_dAOU_1948-2017_monthly.nc'
ds0 = xr.open_dataset(filename)
dT = ds0['dT']
dS = ds0['dS']
dO2 = ds0['dO2']
dAOU = ds0['dAOU']

## calculate Phi components

In [5]:
import traits_db
df = traits_db.df_mi
df.head()

,Species,Phylum,Ao,Ac,Eo
0,Acanthephyra acutifrons,Crustacea,26.809651,13.364732,-0.0278
1,Acanthephyra curtirostris,Crustacea,25.575448,14.364194,0.0380
2,Acanthephyra purpurea,Crustacea,23.310023,7.460879,0.1320
3,Acanthephyra smithi,Crustacea,28.985507,17.657939,-0.0301
5,Acipenser brevirostrum,Chordata,22.123894,4.901716,0.3626


In [6]:
Ac_dist = np.percentile(df.Ac, [50])
#Ao_dist = np.percentile(df.Ao, [25, 50, 75])
Eo_dist = np.percentile(df.Eo, [25, 50, 75])
print(Ac_dist)
#print(Ao_dist)
print(Eo_dist)

[7.34832416]
[0.1985 0.3389 0.5625]


In [7]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product

#for varr in ['O2', 'AOU', 'T', 'S']:
for varr in ['AOU']:
    dsg = xr.Dataset()
    for i, j in product(range(nrow), range(ncol)):
        Ac = Ac_dist[i]
        Eo = Eo_dist[j]
        print(Ac,Eo)
        if varr == 'O2':
            dsg[f'Phi_O2_{nrow*i+j}'] =  ds[f'dPhidO2_{nrow*i+j}'] * dO2 #dEodT=0.022
            print('Phi O2 done')
        elif varr == 'AOU':
            dsg[f'Phi_AOU_{nrow*i+j}'] = -ds[f'dPhidO2_{nrow*i+j}'] * dAOU
            print('Phi AOU done')
        elif varr == 'T':
            dsg[f'Phi_T_{nrow*i+j}'] = ds[f'dPhidT_{nrow*i+j}'] * dT  #dEodT=0.022
            print('Phi T done')
        elif varr == 'S':
            dsg[f'Phi_S_{nrow*i+j}'] = ds[f'dPhidS_{nrow*i+j}'] * dS  #dEodT=0.022
            print('Phi S done')
    dout = f'/glade/scratch/{USER}/DPLE-results'
    os.makedirs(dout, exist_ok=True)
    dsg.load()
    #
    dsg.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_1948-2017_monthly.nc', mode='w')

7.3483241595629805 0.1985
Phi AOU done
7.3483241595629805 0.3389
Phi AOU done
7.3483241595629805 0.5625
Phi AOU done
CPU times: user 3min 33s, sys: 3min 54s, total: 7min 27s
Wall time: 11min 11s
